In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import json
import os
from dotenv import load_dotenv

load_dotenv()

# Establecer la conexión a la base de datos
try:
    connection = mysql.connector.connect(
        host=os.getenv('HOST'),
        user=os.getenv('USERNAME'),
        password=os.getenv('PASSWORD'),
        database=os.getenv('DATABASE')
    )

    if connection.is_connected():
        print("Conexión a MySQL exitosa")

        # Ejecutar la consulta y cargar los resultados en un DataFrame de pandas
        consulta_sql = """
            SELECT TICKET, CLAVE, FECHA_ANALISIS, NO_CONFORMIDAD, REQUIERE_OS, COMENTARIO_ANALISTA, ES_RECURRENTE, MERCADO, CONSUMO_PROMEDIO, MEDIDA, CIRCUITO
            FROM pnrp.bitacora_ac
            WHERE MARCA = 'HEXING'
            AND ESTADO = 'COMPLETADO'
            AND CLAVE IN (
                SELECT CLAVE
                FROM pnrp.bitacora_ac
                WHERE MARCA = 'HEXING' AND ESTADO = 'COMPLETADO'
                AND REQUIERE_OS = 1
                GROUP BY CLAVE
                HAVING COUNT(*) > 2
            );
        """

        # Leer los datos usando pandas
        df = pd.read_sql(consulta_sql, connection)

        # Definir el "system" role único fuera del bucle
        mensaje_system = {
            "role": "system",
            "content": (
                f"Eres un asistente en ingeniería eléctrica que, basándote en comentarios históricos "
                f"de analistas que pueden incluir algunos datos como el número de ticket, la clave del dispositivo, "
                f"la fecha de análisis, la no conformidad identificada, si requiere o no una orden de servicio, "
                f"el comentario del analista, si es recurrente, el mercado al que pertenece, el consumo promedio, "
                f"el tipo de medida, y el circuito, intenta correlacionar e inferir si requiere o no una orden de servicio así como la clasificación de una criticidad, proporciona una justificación "
                f"adecuada en tus respuestas."
            )
        }

        # Crear y escribir en el archivo JSONL
        with open('fine_tuning_data_hexing_si_v1.jsonl', 'w', encoding='utf-8') as archivo:
            for index, row in df.iterrows():
                # Crear estructura de mensaje para el role "user"
                mensajes = [
                    mensaje_system,
                    {
                        "role": "user",
                        "content": (
                            f"¿Requiere la clave {row['CLAVE']} con el ticket {row['TICKET']} del circuito {row['CIRCUITO']}, "
                            f"y fecha de análisis {row['FECHA_ANALISIS']}, <<con la descripción  {row['COMENTARIO_ANALISTA']}>> "
                            f"una orden de servicio? ¿Es recurrente este problema? ¿Cuál es la no conformidad?"
                        )
                    },
                    {
                        "role": "assistant",
                        "content": (
                            f"Reporte de Evaluación:\n"
                            f"Ticket: {row['TICKET']}\n"
                            f"Clave: {row['CLAVE']}\n"
                            f"Circuito: {row['CIRCUITO']}\n"
                            f"Requiere Orden de Servicio: {'Sí' if row['REQUIERE_OS'] == 1 else 'No'}\n"
                            f"No Conformidad: {row['NO_CONFORMIDAD']}\n"
                            f"Es Recurrente: {'Sí' if row['ES_RECURRENTE'] == 1 else 'No'}\n"
                        )
                    }
                ]
                
                # Escribir el conjunto de mensajes en una estructura JSON
                archivo.write(json.dumps({"messages": mensajes}, ensure_ascii=False) + '\n')

        print("Archivo JSONL generado exitosamente.")

except Error as e:
    print(f"Error al conectar con MySQL: {e}")

finally:
    if connection.is_connected():
        connection.close()
        print("Conexión a MySQL cerrada")


In [ ]:
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv

load_dotenv()

connection = None  # Inicializar la variable connection

try:
    # Establecer la conexión
    connection = mysql.connector.connect(
        host=os.getenv('HOST'),
        user=os.getenv('USERNAME'),
        password=os.getenv('PASSWORD'),
    )

    if connection.is_connected():
        print("Conexión a MySQL exitosa")

except Error as e:
    print(f"Error al conectar con MySQL: {e}")

finally:
    if connection and connection.is_connected():  # Verificar si connection no es None
        connection.close()
        print("Conexión a MySQL cerrada")



In [ ]:
#contar tokens en un archivo JSONL

import tiktoken
import json

# Inicializar el tokenizer para el modelo GPT-4 (o el más cercano disponible)
enc = tiktoken.encoding_for_model("gpt-4")

# Ruta del archivo JSONL
ruta_jsonl = 'fine_tuning_data_hexing_970kT_v1.jsonl'

# Contador total de tokens
total_tokens = 0

# Leer y contar los tokens para cada línea en el archivo JSONL
with open(ruta_jsonl, 'r', encoding='utf-8') as archivo:
    for linea in archivo:
        # Parsear la línea como JSON
        datos = json.loads(linea)
        
        # Concatenar todos los contenidos de los mensajes para contar tokens
        texto_completo = ""
        for mensaje in datos.get('messages', []):
            texto_completo += mensaje.get('content', '')
        
        # Contar los tokens en el texto completo
        num_tokens = len(enc.encode(texto_completo.strip()))
        total_tokens += num_tokens

        print(f"Línea procesada, tokens: {num_tokens}")

# Mostrar el número total de tokens en el archivo
print(f"Total de tokens en el archivo JSONL: {total_tokens}")



In [ ]:
import tiktoken

# Inicializar el tokenizador para el modelo GPT-4 (o el más cercano disponible)
enc = tiktoken.encoding_for_model("gpt-4")

# Texto de ejemplo
texto = {
    "messages": [
        {
            "role": "system",
            "content": ("Eres un asistente en ingeniería eléctrica que, basándote en comentarios históricos "
                        "de analistas que pueden incluir algunos datos como el número de ticket, la clave del dispositivo, "
                        "la fecha de análisis, la no conformidad identificada, si requiere o no una orden de servicio, "
                        "el comentario del analista, si es recurrente, el mercado al que pertenece, el consumo promedio, "
                        "el tipo de medida, y el circuito, intenta correlacionar e inferir si requiere o no una orden de servicio así como la clasificación de una criticidad, proporciona una justificación "
                        "adecuada en tus respuestas.")
        },
        {
            "role": "user",
            "content": ("¿Requiere la clave 136764 con el ticket TK-HEX-444 del circuito LPT250, y fecha de análisis 2024-02-05 00:00:00, "
                        "<<con la descripción  El consumo del cliente ha mostrado una tendencia a la baja desde el mes de noviembre de 2023. La última revisión se llevó a cabo el 09/12/2023 a petición del cliente, "
                        "y las cuadrillas encontraron toda la instalación conforme. Dado que se sometió a revisión después de observar la disminución del consumo y aún así no se encontraron irregularidades, "
                        "y considerando que esta revisión fue reciente, no se considera necesario realizar una nueva revisión de la cuenta por el momento.>> una orden de servicio? ¿Es recurrente este problema? ¿Cuál es la no conformidad?")
        },
        {
            "role": "assistant",
            "content": ("Reporte de Evaluación:\n"
                        "Ticket: TK-HEX-444\n"
                        "Clave: 136764\n"
                        "Circuito: LPT250\n"
                        "Requiere Orden de Servicio: No\n"
                        "No Conformidad: None\n"
                        "Es Recurrente: No\n")
        }
    ]
}

# Concatenar todos los contenidos de los mensajes para contar tokens
texto_completo = ""
for mensaje in texto['messages']:
    texto_completo += mensaje['content'] + ""

# Contar los tokens en el texto completo
num_tokens = len(enc.encode(texto_completo.strip()))
print(f"Número de tokens en el texto: {num_tokens}")


In [2]:
import json
import pandas as pd

# Lista de ejemplos con el formato actual
data = pd.read_json('fine_tuning_hexing_consultas.jsonl', lines=True).to_dict(orient='records')

esquema_bd = """
La base de datos tiene las siguientes tablas:

1. **pnrp.airflow_hexing_ulti_comu**: Contiene información sobre comunicaciones de los medidores Hexing. 
    - **CLAVE**: Clave del cliente propietario del medidor.
    - **MEDIDOR**: Identificador del medidor.
    - **FECHA**: Fecha de la comunicación, debería comunicar todos los días a menos que este inactivo.
    - **LECTURA**: Lectura del medidor.

2. **pnrp.airflow_hexing_alarmas**: Contiene información sobre alarmas de los medidores Hexing. Al 
   - **CLAVE**: Clave del cliente propietario del medidor.
   - **MEDIDOR**: Identificador del medidor.
   - **ALARM_CODE**: Código de la alarma.
   - **ALARM_DESC**: Nombre de la alarma.
   - **FECHA**: Fecha de la alarma.

3. **pnrp.airflow_hexing_os**: Órdenes de servicio generadas a los clientes, sus estados y descripciones.
   - **CLAVE**: Clave del cliente propietario del medidor.
   - **OS**: ID de la orden de servicio generada al cliente.
   - **DESCRIPCION_OS**: Descripción de la orden de servicio.
   - **ESTADO**: Estado de la orden de servicio. Estados posibles: RESUELTA, EJECUTADA EN TERRENO, ENVIAR A CAMPO, PROGRAMADA. Se debe hacer trim para evitar espacios en blanco en la tabla.
   - **CATEGORIA**: Categoría de la orden de servicio.
   - **DESCRIPCION**: Descripción adicional sobre la orden de servicio.
   - **FECHA_GENERADA**: Fecha de generación de la orden de servicio.
   - **FECHA_EJECUCION**: Fecha de ejecución de la orden de servicio.


4. **pnrp.bitacora_ac**: Contiene registros de tickets de orden de servicio, análisis y otros detalles.
   - **ID**: Identificador único del registro.
   - **TICKET**: Número de ticket.
   - **CLAVE**: Clave del cliente propietario del medidor.
   - **MEDIDOR**: Identificador del medidor.
   - **MARCA**: Marca del medidor (HEXING, UNION, ELSTER).
   - **PERIODO_INICIAL**: Fecha de inicio del periodo de análisis.
   - **PERIODO_FINAL**: Fecha de finalización del periodo de análisis.
   - **NOMBRE**: Nombre del cliente asociado al medidor.
   - **ANALISTA**: Nombre del analista que realizó el análisis.
   - **FECHA_ASIGNACION**: Fecha en que se asignó el ticket al analista.
   - **FECHA_ANALISIS**: Fecha en que se realizó el análisis del ticket.
   - **FECHA_REGISTRO**: Fecha en que se registró el ticket.
   - **DIAS_ULT_REVISION**: Días transcurridos desde la última revisión del medidor.
   - **ALARMA**: Descripción de la alarma de referencia del ticket.
   - **FECHA_ALARMA**: Fecha en que se generó la alarma referencia.
   - **NO_CONFORMIDAD**: No conformidad identificada durante el análisis.
   - **REQUIERE_OS**: Indica si requiere orden de servicio (0 o 1).
   - **COMENTARIO_ANALISTA**: Comentarios proporcionados por el analista.
   - **ES_RECURRENTE**: Indica si al cliente ya se le han generado tickets anteriormente (0 o 1).
   - **APROBACION_ANALISIS**: Estado de aprobación del análisis.
   - **COMENTARIO_R5**: Comentarios adicionales de región 5 (una región de la empresa).
   - **OS**: Orden de servicio relacionada.
   - **FECHA_CREACION_OS**: Fecha de creación de la orden de servicio.
   - **ANOMALIA**: Anomalía reportada en el medidor.
   - **FECHA_ANOMALIA_INICIAL**: Fecha de inicio de la anomalía.
   - **FECHA_ANOMALIA_FINAL**: Fecha de finalización de la anomalía.
   - **CAMBIO_MEDIDOR**: Indica si hubo un cambio de medidor.
   - **SERIE_NUEVO_MEDIDOR**: Número de serie del nuevo medidor.
   - **COMENTARIO_CENTRAL**: Comentarios del área central (área central del departamento de Telegestión de la empresa) sobre el ticket.
   - **DIRECCION**: Dirección del cliente.
   - **MULTIPLICADOR**: Multiplicador del medidor.
   - **MERCADO**: Tipo de mercado al que pertenece el cliente.
   - **CONSUMO_PROMEDIO**: Consumo promedio del cliente.
   - **TARIFA**: Tarifa asignada al cliente.
   - **SECTOR**: Sector al que pertenece el cliente.
   - **MEDIDA**: Tipo de medida (e.g., directa, indirecta).
   - **LATITUD**: Latitud de la ubicación del medidor.
   - **LONGITUD**: Longitud de la ubicación del medidor.
   - **CIRCUITO**: Circuito eléctrico al que pertenece el medidor.
   - **CRITICIDAD_ALARMA**: Nivel de criticidad de la alarma generada.
   - **PONDERACION_CRITICIDAD**: Ponderación de la criticidad asignada a la alarma.
   - **ESTADO**: Estado del ticket. Estados posibles: COMPLETADO, ANULADO, REASIGNADO, EN PROCESO, PENDIENTE.
   - **REGION_ASIGNACION**: Región a la que se asigna el ticket.

5. **pnrp.airflow_hexing_universo**: Contiene registros de los medidores Hexing, junto con información detallada de cada medidor y su ubicación.
   - **MARCA_MEDIDOR**: Marca del medidor, siempre es HEXING.
   - **CLAVE_CATALOGO**: Clave de cliente asociada al catálogo del medidor.
   - **MEDIDOR_CATALOGO**: Identificador del medidor dentro del catálogo.
   - **RTP**: Registro de potencia activa.
   - **RTC**: Registro de tiempo de conexión.
   - **RTPxRTC**: Producto entre el RTP y el RTC.
   - **CLAVE_INCMS**: Clave del cliente en el sistema INCMS.
   - **NOMBRE_ABONADO_INCMS**: Nombre del abonado en el sistema INCMS.
   - **MEDIDOR_INCMS**: Identificador del medidor en el sistema INCMS.
   - **DIRECCION_INCMS**: Dirección registrada del abonado en el sistema INCMS.
   - **COMPARACION**: Indica si los datos del medidor coinciden entre los sistemas.
   - **MULTIPLICADOR_INCMS**: Multiplicador del medidor en el sistema INCMS.
   - **CODIGO_LECTURA**: Código relacionado con la lectura del medidor.
   - **LECTURA_ACTUAL**: Última lectura registrada del medidor.
   - **LECTURA_ANTERIOR**: Lectura previa del medidor.
   - **ULTIMO_CONSUMO**: Consumo reportado en el último ciclo de facturación.
   - **ULT_MES_FACT**: Último mes facturado para el medidor.
   - **DIAS_FACTURADOS**: Cantidad de días facturados en el último ciclo.
   - **TARIFA**: Tarifa aplicada al abonado.
   - **POTENCIA_ACTIVA_PROMEDIO**: Potencia activa promedio registrada.
   - **TIPO_MEDIDA**: Tipo de medida (DIRECTA, INDIRECTA, SEMI DIRECTA).
   - **MERCADO**: Mercado al que pertenece el abonado (por ejemplo: ALTOS_CONSUMIDORES, GOBIERNO).
   - **REGION_PNRP**: Región de PNRP en la que está ubicado el abonado (Corresponden a ciudades de Honduras).
   - **ASIGNADO**: Región o equipo asignado para la revisión o gestión del medidor.
   - **ZONA**: Zona geográfica del abonado.
   - **COORD_X**: Coordenada X de la ubicación del medidor.
   - **COORD_Y**: Coordenada Y de la ubicación del medidor.
   - **CIRCUITO**: Circuito eléctrico al que pertenece el medidor.
   - **SUBESTACION**: Subestación asociada al medidor.
   - **COORD_U_X**: Coordenada X de la subestación.
   - **COORD_U_Y**: Coordenada Y de la subestación.


NOTA: Las solo tienen las columnas mostradas pero las 3 tablas se pueden relacionar por la clave y el medidor, excepto airflow_hexing_os que solo se puede unir por clave, esto para que puedas hacer joins y obtener la información necesaria. 
"""

system_message = f"""
Eres un asistente especializado en generación de consultas SQL a partir de lenguaje natural.
        El esquema de la base de datos es el siguiente:
        {esquema_bd}
        
        Convierte la siguiente consulta en lenguaje natural a SQL para una base de datos MySQL. 
        Genera únicamente el código SQL, sin comentarios, ni saltos de línea, ni marcas de código adicionales.

        Es de suma importancia que si la pregunta del usuario no tiene que ver con nuestro contexto de análisis de medidores Hexing, retorna la consulta: SELECT "No puedo responder a esa pregunta" as ERROR. Si te piden DELETE, UPDATE, DROP o algo que pueda afectar la base de datos, retorna la consulta: SELECT "No puedo ejecutar esa acción" as ERROR.
        - Al consultar airflow_hexing_os, siempre que se consulte sobre OS se debe regresar el número de la OS, incluso si no lo especifica la pregunta.
        - Al consultar bitacora_ac, siempre que se pida tickets se debe regresar el número del ticket, incluso si no lo especifica la pregunta.
        - Al consultar estados, siempre debes hacer TRIM para evitar espacios en blanco en la tabla.
        - La tabla airflow_hexing_alarmas siempre se ordena por fecha de forma descendente y se consulta con limit 20, a menos que se especifique un límite diferente o un rango de fechas en la pregunta del usuario.
        - La tabla airflow_hexing_os nunca se le debe hacer un LIMIT.
        - Nunca filtres por estado a menos que se pida en la pregunta.
"""

# Convertir el formato
def convert_format(data):
    converted = []
    for item in data:
        entry = {
            "messages": [
                {"role": "system", "content": system_message},
                {"role": "user", "content": item['prompt']},
                {"role": "assistant", "content": item['completion']}
            ]
        }
        converted.append(entry)
    return converted

# Convertir los datos
converted_data = convert_format(data)

# Guardar los datos en un archivo JSONL
with open('data_fine_tuning_hexing_consultas_v2.jsonl', 'w', encoding='utf-8') as f:
    for entry in converted_data:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')




In [6]:
import pandas as pd
import json


# Especifica los nombres de archivo
jsonl_file = 'fine_tuning_2.jsonl'  # Reemplaza con tu archivo JSONL
csv_file = 'fine_tuning_gemini.csv'     # Nombre del archivo de salida CSV

jsonl = pd.read_json(jsonl_file, lines=True)

# agregar columna system
jsonl['system'] = """Eres un asistente especializado en generación de consultas SQL a partir de lenguaje natural. El esquema de la base de datos es el siguiente:
La base de datos tiene las siguientes tablas:

1. **pnrp.airflow_hexing_ulti_comu**: Contiene información sobre comunicaciones de los medidores Hexing. 
    - **CLAVE**: Clave del cliente propietario del medidor.
    - **MEDIDOR**: Identificador del medidor.
    - **FECHA**: Fecha de la comunicación, debería comunicar todos los días a menos que este inactivo.
    - **LECTURA**: Lectura del medidor.

2. **pnrp.airflow_hexing_alarmas**: Contiene información sobre alarmas de los medidores Hexing. Al 
   - **CLAVE**: Clave del cliente propietario del medidor.
   - **MEDIDOR**: Identificador del medidor.
   - **ALARM_CODE**: Código de la alarma.
   - **ALARM_DESC**: Nombre de la alarma.
   - **FECHA**: Fecha de la alarma.

3. **pnrp.airflow_hexing_os**: Órdenes de servicio generadas a los clientes, sus estados y descripciones.
   - **CLAVE**: Clave del cliente propietario del medidor.
   - **OS**: ID de la orden de servicio generada al cliente.
   - **DESCRIPCION_OS**: Descripción de la orden de servicio.
   - **ESTADO**: Estado de la orden de servicio. Estados posibles: RESUELTA, EJECUTADA EN TERRENO, ENVIAR A CAMPO, PROGRAMADA. Se debe hacer trim para evitar espacios en blanco en la tabla.
   - **CATEGORIA**: Categoría de la orden de servicio.
   - **DESCRIPCION**: Descripción adicional sobre la orden de servicio.
   - **FECHA_GENERADA**: Fecha de generación de la orden de servicio.
   - **FECHA_EJECUCION**: Fecha de ejecución de la orden de servicio.


4. **pnrp.bitacora_ac**: Contiene registros de tickets de orden de servicio, análisis y otros detalles.
   - **ID**: Identificador único del registro.
   - **TICKET**: Número de ticket.
   - **CLAVE**: Clave del cliente propietario del medidor.
   - **MEDIDOR**: Identificador del medidor.
   - **MARCA**: Marca del medidor (HEXING, UNION, ELSTER).
   - **PERIODO_INICIAL**: Fecha de inicio del periodo de análisis.
   - **PERIODO_FINAL**: Fecha de finalización del periodo de análisis.
   - **NOMBRE**: Nombre del cliente asociado al medidor.
   - **ANALISTA**: Nombre del analista que realizó el análisis.
   - **FECHA_ASIGNACION**: Fecha en que se asignó el ticket al analista.
   - **FECHA_ANALISIS**: Fecha en que se realizó el análisis del ticket.
   - **FECHA_REGISTRO**: Fecha en que se registró el ticket.
   - **DIAS_ULT_REVISION**: Días transcurridos desde la última revisión del medidor.
   - **ALARMA**: Descripción de la alarma de referencia del ticket.
   - **FECHA_ALARMA**: Fecha en que se generó la alarma referencia.
   - **NO_CONFORMIDAD**: No conformidad identificada durante el análisis.
   - **REQUIERE_OS**: Indica si requiere orden de servicio (0 o 1).
   - **COMENTARIO_ANALISTA**: Comentarios proporcionados por el analista.
   - **ES_RECURRENTE**: Indica si al cliente ya se le han generado tickets anteriormente (0 o 1).
   - **APROBACION_ANALISIS**: Estado de aprobación del análisis.
   - **COMENTARIO_R5**: Comentarios adicionales de región 5 (una región de la empresa).
   - **OS**: Orden de servicio relacionada.
   - **FECHA_CREACION_OS**: Fecha de creación de la orden de servicio.
   - **ANOMALIA**: Anomalía reportada en el medidor.
   - **FECHA_ANOMALIA_INICIAL**: Fecha de inicio de la anomalía.
   - **FECHA_ANOMALIA_FINAL**: Fecha de finalización de la anomalía.
   - **CAMBIO_MEDIDOR**: Indica si hubo un cambio de medidor.
   - **SERIE_NUEVO_MEDIDOR**: Número de serie del nuevo medidor.
   - **COMENTARIO_CENTRAL**: Comentarios del área central (área central del departamento de Telegestión de la empresa) sobre el ticket.
   - **DIRECCION**: Dirección del cliente.
   - **MULTIPLICADOR**: Multiplicador del medidor.
   - **MERCADO**: Tipo de mercado al que pertenece el cliente.
   - **CONSUMO_PROMEDIO**: Consumo promedio del cliente.
   - **TARIFA**: Tarifa asignada al cliente.
   - **SECTOR**: Sector al que pertenece el cliente.
   - **MEDIDA**: Tipo de medida (e.g., directa, indirecta).
   - **LATITUD**: Latitud de la ubicación del medidor.
   - **LONGITUD**: Longitud de la ubicación del medidor.
   - **CIRCUITO**: Circuito eléctrico al que pertenece el medidor.
   - **CRITICIDAD_ALARMA**: Nivel de criticidad de la alarma generada.
   - **PONDERACION_CRITICIDAD**: Ponderación de la criticidad asignada a la alarma.
   - **ESTADO**: Estado del ticket. Estados posibles: COMPLETADO, ANULADO, REASIGNADO, EN PROCESO, PENDIENTE.
   - **REGION_ASIGNACION**: Región a la que se asigna el ticket.

5. **pnrp.airflow_hexing_universo**: Contiene registros de los medidores Hexing, junto con información detallada de cada medidor y su ubicación.
   - **MARCA_MEDIDOR**: Marca del medidor, siempre es HEXING.
   - **CLAVE_CATALOGO**: Clave de cliente asociada al catálogo del medidor.
   - **MEDIDOR_CATALOGO**: Identificador del medidor dentro del catálogo.
   - **RTP**: Registro de potencia activa.
   - **RTC**: Registro de tiempo de conexión.
   - **RTPxRTC**: Producto entre el RTP y el RTC.
   - **CLAVE_INCMS**: Clave del cliente en el sistema INCMS.
   - **NOMBRE_ABONADO_INCMS**: Nombre del abonado en el sistema INCMS.
   - **MEDIDOR_INCMS**: Identificador del medidor en el sistema INCMS.
   - **DIRECCION_INCMS**: Dirección registrada del abonado en el sistema INCMS.
   - **COMPARACION**: Indica si los datos del medidor coinciden entre los sistemas.
   - **MULTIPLICADOR_INCMS**: Multiplicador del medidor en el sistema INCMS.
   - **CODIGO_LECTURA**: Código relacionado con la lectura del medidor.
   - **LECTURA_ACTUAL**: Última lectura registrada del medidor.
   - **LECTURA_ANTERIOR**: Lectura previa del medidor.
   - **ULTIMO_CONSUMO**: Consumo reportado en el último ciclo de facturación.
   - **ULT_MES_FACT**: Último mes facturado para el medidor.
   - **DIAS_FACTURADOS**: Cantidad de días facturados en el último ciclo.
   - **TARIFA**: Tarifa aplicada al abonado.
   - **POTENCIA_ACTIVA_PROMEDIO**: Potencia activa promedio registrada.
   - **TIPO_MEDIDA**: Tipo de medida (DIRECTA, INDIRECTA, SEMI DIRECTA).
   - **MERCADO**: Mercado al que pertenece el abonado (por ejemplo: ALTOS_CONSUMIDORES, GOBIERNO).
   - **REGION_PNRP**: Región de PNRP en la que está ubicado el abonado (Corresponden a ciudades de Honduras).
   - **ASIGNADO**: Región o equipo asignado para la revisión o gestión del medidor.
   - **ZONA**: Zona geográfica del abonado.
   - **COORD_X**: Coordenada X de la ubicación del medidor.
   - **COORD_Y**: Coordenada Y de la ubicación del medidor.
   - **CIRCUITO**: Circuito eléctrico al que pertenece el medidor.
   - **SUBESTACION**: Subestación asociada al medidor.
   - **COORD_U_X**: Coordenada X de la subestación.
   - **COORD_U_Y**: Coordenada Y de la subestación.


NOTA: Las solo tienen las columnas mostradas pero las 3 tablas se pueden relacionar por la clave y el medidor, excepto airflow_hexing_os que solo se puede unir por clave, esto para que puedas hacer joins y obtener la información necesaria. 

Convierte la siguiente consulta en lenguaje natural a SQL para una base de datos MySQL. 
        Genera únicamente el código SQL, sin comentarios, ni saltos de línea, ni marcas de código adicionales.

        Es de suma importancia que si la pregunta del usuario no tiene que ver con nuestro contexto de análisis de medidores Hexing, retorna la consulta: SELECT "No puedo responder a esa pregunta" as ERROR. Si te piden DELETE, UPDATE, DROP o algo que pueda afectar la base de datos, retorna la consulta: SELECT "No puedo ejecutar esa acción" as ERROR.
        - Al consultar airflow_hexing_os, siempre que se consulte sobre OS se debe regresar el número de la OS, incluso si no lo especifica la pregunta.
        - Al consultar bitacora_ac, siempre que se pida tickets se debe regresar el número del ticket, incluso si no lo especifica la pregunta.
        - Al consultar estados, siempre debes hacer TRIM para evitar espacios en blanco en la tabla.
        - La tabla airflow_hexing_alarmas siempre se ordena por fecha de forma descendente y se consulta con limit 20, a menos que se especifique un límite diferente o un rango de fechas en la pregunta del usuario.
        - La tabla airflow_hexing_os nunca se le debe hacer un LIMIT.
        - Nunca filtres por estado a menos que se pida en la pregunta.
""".replace('\n', '. ')

jsonl.to_csv(csv_file, index=False)


